In [1]:
#import packages
import pandas as pd
import geopandas as gpd
import requests
from shapely.geometry import shape, Point
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import urllib.request, json
import requests
import urllib.parse
import datetime
import os
import dask.dataframe as dd
import calendar
from datetime import datetime, timedelta

from geopandas.tools import sjoin

import warnings
warnings.filterwarnings('ignore')

/Users/lizjohnson/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [213]:
#import nta shapefile
nta_url = 'https://data.cityofnewyork.us/resource/q2z5-ai38.json'

query = (nta_url+'?'
        "$select=*"
        "&$limit=10000")
query = query.replace(" ", "%20")
response = urllib.request.urlopen(query)
data = json.loads(response.read())


for d in data:
    d['geometry'] = shape(d['the_geom'])

#store in a geodataframe
nta_nyc = gpd.GeoDataFrame(data, geometry = 'geometry', crs = 'EPSG: 4326')
nta_nyc = nta_nyc.drop(columns = ['the_geom'])
nta_nyc.head()

,boro_code,boro_name,county_fips,ntacode,ntaname,shape_leng,shape_area,geometry
0,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ..."
1,2,Bronx,005,BX28,Van Cortlandt Village,21945.719299,25666124.5948,"MULTIPOLYGON (((-73.88705 40.88435, -73.88705 ..."
2,4,Queens,081,QN55,South Ozone Park,36708.1693055,82461393.7368,"MULTIPOLYGON (((-73.80577 40.68293, -73.80552 ..."
3,3,Brooklyn,047,BK40,Windsor Terrace,19033.6720656,14041667.99,"MULTIPOLYGON (((-73.98017 40.66115, -73.98021 ..."
4,3,Brooklyn,047,BK50,Canarsie,43703.6096661,82089678.6389,"MULTIPOLYGON (((-73.88834 40.64671, -73.88835 ..."


In [331]:
#import streeteast median sales data
streeteasy = pd.read_csv('../Data/medianSalesPrice_All.csv')
streeteasy.head(15)

,areaName,Borough,areaType,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,...,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
0,All Downtown,Manhattan,submarket,900000.0,849500.0,990000.0,992793.0,916962.5,800000.0,1125000.0,...,1625000.0,1399999.0,1400178.0,1400000.0,1345000.0,1450000.0,1350000.0,1399420.0,1295000.0,1550000.0
1,All Midtown,Manhattan,submarket,685000.0,637500.0,600000.0,687500.0,750000.0,598000.0,748000.0,...,875000.0,875000.0,875000.0,780000.0,795000.0,1045000.0,887500.0,800000.0,800000.0,875000.0
2,All Upper East Side,Manhattan,submarket,1004500.0,831421.0,995000.0,875000.0,1200000.0,827391.0,993750.0,...,1150000.0,1242500.0,1132155.0,1192500.0,997000.0,1114900.0,1237500.0,1207500.0,1400000.0,1145000.0
3,All Upper Manhattan,Manhattan,submarket,366022.0,393496.0,350139.5,334620.0,338750.0,415740.0,433000.0,...,725000.0,640000.0,685000.0,680000.0,592000.0,495930.0,457500.0,547500.0,659000.0,730000.0
4,All Upper West Side,Manhattan,submarket,860421.0,875000.0,747250.0,803603.5,900000.0,769750.0,985000.0,...,1287500.0,1270000.0,1265000.0,1317500.0,1120000.0,975000.0,1047500.0,1072441.5,1100000.0,967500.0
5,Astoria,Queens,neighborhood,402000.0,605237.5,380000.0,415000.0,392660.0,400000.0,380000.0,...,936776.0,883500.0,620000.0,810000.0,1049000.0,999000.0,980000.0,950000.0,1150000.0,927500.0
6,Auburndale,Queens,neighborhood,670000.0,520000.0,NaN,549250.0,575000.0,533250.0,554250.0,...,912500.0,940000.0,1053000.0,990000.0,925000.0,982500.0,910000.0,915000.0,NaN,NaN
7,Bath Beach,Brooklyn,neighborhood,660000.0,353000.0,498888.0,305000.0,NaN,537500.0,NaN,...,852500.0,1052500.0,1112500.0,744500.0,1050000.0,NaN,NaN,1085000.0,NaN,NaN
8,Battery Park City,Manhattan,neighborhood,NaN,502650.0,763000.0,2031917.0,1476462.5,NaN,NaN,...,1002500.0,885000.0,1192487.5,1394800.0,1225880.0,1575000.0,1466250.0,1357500.0,1687500.0,NaN
9,Bay Ridge,Brooklyn,neighborhood,480000.0,377500.0,270732.5,624500.0,260000.0,287500.0,355000.0,...,765000.0,999000.0,599500.0,650000.0,625000.0,667000.0,833946.0,996866.0,435000.0,900000.0


In [362]:
neigh_salesprice = streeteasy[(streeteasy['areaType'] == 'neighborhood')&(streeteasy['Borough'] != 'Staten Island')].drop(columns = ['areaType','Borough']).set_index('areaName')
neigh_salesprice = neigh_salesprice.dropna(axis = 0,thresh=len(neigh_salesprice.columns)/2).fillna(0).reset_index()
neigh_salesprice.head()

,areaName,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,...,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
0,Astoria,402000.0,605237.5,380000.0,415000.0,392660.0,400000.0,380000.0,555018.5,425880.0,...,936776.0,883500.0,620000.0,810000.0,1049000.0,999000.0,980000.0,950000.0,1150000.0,927500.0
1,Auburndale,670000.0,520000.0,0.0,549250.0,575000.0,533250.0,554250.0,639500.0,546000.0,...,912500.0,940000.0,1053000.0,990000.0,925000.0,982500.0,910000.0,915000.0,0.0,0.0
2,Bath Beach,660000.0,353000.0,498888.0,305000.0,0.0,537500.0,0.0,0.0,645000.0,...,852500.0,1052500.0,1112500.0,744500.0,1050000.0,0.0,0.0,1085000.0,0.0,0.0
3,Battery Park City,0.0,502650.0,763000.0,2031917.0,1476462.5,0.0,0.0,1165000.0,1247356.0,...,1002500.0,885000.0,1192487.5,1394800.0,1225880.0,1575000.0,1466250.0,1357500.0,1687500.0,0.0
4,Bay Ridge,480000.0,377500.0,270732.5,624500.0,260000.0,287500.0,355000.0,397500.0,572473.0,...,765000.0,999000.0,599500.0,650000.0,625000.0,667000.0,833946.0,996866.0,435000.0,900000.0


In [336]:
len(neigh_salesprice)

103

In [337]:
len(neigh_salesprice.index.unique().tolist())

103

In [338]:
streeteasy_neighborhoods = neigh_salesprice.index.unique().tolist()

In [339]:
streeteasy_neighborhoods

['Astoria',
 'Auburndale',
 'Bath Beach',
 'Battery Park City',
 'Bay Ridge',
 'Bayside',
 'Bedford-Stuyvesant',
 'Bellerose',
 'Bensonhurst',
 'Borough Park',
 'Briarwood',
 'Brighton Beach',
 'Brooklyn Heights',
 'Brownsville',
 'Bushwick',
 'Cambria Heights',
 'Canarsie',
 'Carroll Gardens',
 'Central Harlem',
 'Chelsea',
 'Clearview',
 'Clinton Hill',
 'College Point',
 'Coney Island',
 'Corona',
 'Crown Heights',
 'Ditmas Park',
 'Douglaston',
 'Downtown Brooklyn',
 'Dyker Heights',
 'East Flatbush',
 'East New York',
 'East Village',
 'Elmhurst',
 'Financial District',
 'Flatbush',
 'Flatiron',
 'Flatlands',
 'Flushing',
 'Forest Hills',
 'Fort Greene',
 'Glen Oaks',
 'Glendale',
 'Gramercy Park',
 'Gravesend',
 'Greenpoint',
 'Greenwich Village',
 'Hollis',
 'Howard Beach',
 'Jackson Heights',
 'Jamaica',
 'Jamaica Estates',
 'Kensington',
 'Kew Gardens',
 'Kew Gardens Hills',
 'Laurelton',
 'Little Neck',
 'Long Island City',
 'Lower East Side',
 'Marine Park',
 'Maspeth',
 'Mi

In [340]:
ntas = nta_nyc['ntaname'][nta_nyc['boro_name']!='Staten Island'].tolist()
len(ntas)

176

In [341]:
ntas.sort()
ntas

['Airport',
 'Allerton-Pelham Gardens',
 'Astoria',
 'Auburndale',
 'Baisley Park',
 'Bath Beach',
 'Battery Park City-Lower Manhattan',
 'Bay Ridge',
 'Bayside-Bayside Hills',
 'Bedford',
 'Bedford Park-Fordham North',
 'Bellerose',
 'Belmont',
 'Bensonhurst East',
 'Bensonhurst West',
 'Borough Park',
 'Breezy Point-Belle Harbor-Rockaway Park-Broad Channel',
 'Briarwood-Jamaica Hills',
 'Brighton Beach',
 'Bronxdale',
 'Brooklyn Heights-Cobble Hill',
 'Brownsville',
 'Bushwick North',
 'Bushwick South',
 'Cambria Heights',
 'Canarsie',
 'Carroll Gardens-Columbia Street-Red Hook',
 'Central Harlem North-Polo Grounds',
 'Central Harlem South',
 'Chinatown',
 'Claremont-Bathgate',
 'Clinton',
 'Clinton Hill',
 'Co-op City',
 'College Point',
 'Corona',
 'Crotona Park East',
 'Crown Heights North',
 'Crown Heights South',
 'Cypress Hills-City Line',
 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill',
 'Douglas Manor-Douglaston-Little Neck',
 'Dyker Heights',
 'East Concourse-Concourse V

In [223]:
neigh_nta_dict = {'Astoria': ['Astoria','Old Astoria','Steinway'],
                  'Auburndale': 'Auburndale',
                  'Bath Beach': 'Bath Beach',
                  'Battery Park City': 'Battery Park City-Lower Manhattan',
                  'Bay Ridge': 'Bay Ridge',
                  'Baychester': 'Eastchester-Edenwald-Baychester',
                  'Bayside': 'Bayside-Bayside Hills',
                  'Bedford Park': 'Bedford Park-Fordham North',
                  'Bedford-Stuyvesant': ['Bedford','Stuyvesant Heights'],
                  'Bellerose': 'Bellerose',
                  'Belmont': 'Belmont',
                  'Bensonhurst': ['Bensonhurst East','Bensonhurst West'],
                  'Bergen Beach':
                  'Georgetown-Marine Park-Bergen Beach-Mill Basin',
                  'Boerum Hill': 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill',
                  'Borough Park': 'Borough Park',
                  'Briarwood': 'Briarwood-Jamaica Hills',
                  'Brighton Beach': ['Brighton Beach','West Brighton'],
                  'Bronxwood': 'Bronxdale',
                  'Brooklyn Heights': 'Brooklyn Heights-Cobble Hill',
                  'Brookville': 'Springfield Gardens South-Brookville',
                  'Brownsville': 'Brownsville', 
                  'Bushwick': ['Bushwick North','Bushwick South'],
                  'Cambria Heights': 'Cambria Heights',
                  'Canarsie': 'Canarsie',
                  'Carroll Gardens': 'Carroll Gardens-Columbia Street-Red Hook',
                  'Castle Hill': 'Soundview-Castle Hill-Clason Point-Harding Park',
                  'Central Harlem': ['Central Harlem North-Polo Grounds','Central Harlem South'],
                  'Central Park South': 'Midtown-Midtown South',
                  'Chelsea': 'Hudson Yards-Chelsea-Flatiron-Union Square',
                  'Chinatown': 'Chinatown',
                  'City Island': 'Pelham Bay-Country Club-City Island',
                  'Civic Center': 'SoHo-TriBeCa-Civic Center-Little Italy',
                  'Clearview': 'Ft. Totten-Bay Terrace-Clearview',
                  'Clinton Hill': 'Clinton Hill',
                  'Co-op City': 'Co-op City',
                  'Cobble Hill': 'Brooklyn Heights-Cobble Hill',
                  'College Point': 'College Point',
                  'Columbia St Waterfront District':'Carroll Gardens-Columbia Street-Red Hook',
                  'Concourse': ['West Concourse','East Concourse-Concourse Village'],
                  'Coney Island': 'Seagate-Coney Island',
                  'Corona': 'Corona',
                  'Country Club': 'Pelham Bay-Country Club-City Island',
                  'Crotona Park East': 'Crotona Park East',
                  'Crown Heights': ['Ocean Hill','Crown Heights North','Crown Heights South'],
                  'DUMBO': 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill', 
                  'Ditmas Park': 'Flatbush',
                  'Douglaston': 'Douglas Manor-Douglaston-Little Neck', 
                  'Downtown Brooklyn': 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill',
                  'Dyker Heights': 'Dyker Heights', 
                  'East Elmhurst': 'East Elmhurst',
                  'East Flatbush': 'East Flatbush-Farragut',
                  'East Harlem': ['East Harlem North','East Harlem South'], 
                  'East New York': ['East New York','East New York (Pennsylvania Ave)','Cypress Hills-City Line','Starrett City'],
                  'East Tremont': 'East Tremont',
                  'East Village': 'East Village',
                  'Eastchester': 'Eastchester-Edenwald-Baychester',
                  'Edenwald': 'Eastchester-Edenwald-Baychester', 
                  'Elmhurst': ['Elmhurst','Elmhurst-Maspeth'],
                  'Financial District': 'Battery Park City-Lower Manhattan',
                  'Flatbush': ['Flatbush','Erasmus'],
                  'Flatiron': 'Hudson Yards-Chelsea-Flatiron-Union Square',
                  'Flatlands': 'Flatlands',
                  'Floral Park': 'Glen Oaks-Floral Park-New Hyde Park',
                  'Flushing': ['Flushing','Murray Hill','East Flushing','Queensboro Hill'],
                  'Fordham': 'Bedford Park-Fordham North',
                  'Forest Hills': 'Forest Hills',
                  'Fort Greene': 'Fort Greene',
                  'Fresh Meadows': 'Fresh Meadows-Utopia',
                  'Gerritsen Beach': 'Sheepshead Bay-Gerritsen Beach-Manhattan Beach',
                  'Glen Oaks': 'Glen Oaks-Floral Park-New Hyde Park',
                  'Glendale': 'Glendale',
                  'Gowanus': 'Park Slope-Gowanus',
                  'Gramercy Park': 'Gramercy',
                  'Gravesend': 'Gravesend',
                  'Greenpoint': 'Greenpoint',
                  'Greenwich Village': 'West Village',
                  'Greenwood': 'Sunset Park West',
                  'Hamilton Heights': 'Hamilton Heights',
                  'Highbridge': 'Highbridge',
                  'Hillcrest': 'Pomonok-Flushing Heights-Hillcrest',
                  'Hollis': 'Hollis', 
                  'Howard Beach': 'Lindenwood-Howard Beach',
                  'Hunts Point': 'Hunts Point',
                  'Inwood': 'Marble Hill-Inwood',
                  'Jackson Heights': 'Jackson Heights',
                  'Jamaica': 'Jamaica',
                  'Jamaica Estates': 'Jamaica Estates-Holliswood',
                  'Jamaica Hills': 'Briarwood-Jamaica Hills',
                  'Kensington': 'Kensington-Ocean Parkway',
                  'Kew Gardens': 'Kew Gardens',
                  'Kew Gardens Hills': 'Kew Gardens Hills', 
                  'Kingsbridge': 'Van Cortlandt Village',
                  'Laconia': 'Allerton-Pelham Gardens',
                  'Laurelton': 'Laurelton',
                  'Little Italy': 'SoHo-TriBeCa-Civic Center-Little Italy',
                  'Little Neck': 'Douglas Manor-Douglaston-Little Neck',
                  'Long Island City': 'Queensbridge-Ravenswood-Long Island City',
                  'Longwood': 'Longwood',
                  'Lower East Side': 'Lower East Side',
                  'Manhattan Beach': 'Sheepshead Bay-Gerritsen Beach-Manhattan Beach',
                  'Marble Hill': 'Marble Hill-Inwood',
                  'Marine Park': 'Georgetown-Marine Park-Bergen Beach-Mill Basin',
                  'Maspeth': 'Maspeth',
                  'Melrose': ['Morrisania-Melrose','Melrose South-Mott Haven North'],
                  'Middle Village': 'Middle Village', 
                  'Midtown': 'Midtown-Midtown South',
                  'Midtown East': ['Turtle Bay-East Midtown','Murray Hill-Kips Bay'], 
                  'Midtown South': 'Midtown-Midtown South',
                  'Midtown West': 'Clinton',
                  'Midwood': 'Midwood',
                  'Mill Basin': 'Georgetown-Marine Park-Bergen Beach-Mill Basin',
                  'Morningside Heights': 'Morningside Heights',
                  'Morris Heights': 'University Heights-Morris Heights',
                  'Morris Park': 'Van Nest-Morris Park-Westchester Square',
                  'Morrisania': 'Morrisania-Melrose',
                  'Mott Haven': 'Melrose South-Mott Haven North',
                  'New Hyde Park': 'Glen Oaks-Floral Park-New Hyde Park',
                  'Nolita': 'SoHo-TriBeCa-Civic Center-Little Italy',
                  'North Corona': 'North Corona',
                  'Norwood': 'Norwood',
                  'Oakland Gardens': 'Oakland Gardens',
                  'Ocean Parkway': 'Ocean Parkway South',
                  'Old Mill Basin': 'Georgetown-Marine Park-Bergen Beach-Mill Basin',
                  'Ozone Park': 'Ozone Park',
                  'Park Slope': 'Park Slope-Gowanus',
                  'Parkchester': 'Parkchester',
                  'Pelham Bay': 'Pelham Bay-Country Club-City Island',
                  'Pelham Gardens': 'Allerton-Pelham Gardens',
                  'Pelham Parkway': 'Pelham Parkway',
                  'Pomonok': 'Pomonok-Flushing Heights-Hillcrest',
                  'Port Morris': 'Mott Haven-Port Morris',
                  'Prospect Heights': 'Prospect Heights',
                  'Prospect Lefferts Gardens': 'Prospect Lefferts Gardens-Wingate', 
                  'Prospect Park South': 'Flatbush',
                  'Queens Village': 'Queens Village',
                  'Red Hook': 'Carroll Gardens-Columbia Street-Red Hook',
                  'Rego Park': 'Rego Park',
                  'Richmond Hill': 'Richmond Hill',
                  'Ridgewood': 'Ridgewood',
                  'Riverdale': ['Spuyten Duyvil-Kingsbridge','North Riverdale-Fieldston-Riverdale'],
                  'Rockaway All': ['Far Rockaway-Bayswater','Hammels-Arverne-Edgemere','Breezy Point-Belle Harbor-Rockaway Park-Broad Channel'],
                  'Roosevelt Island': 'Lenox Hill-Roosevelt Island',
                  'Rosedale': 'Rosedale',
                  'Schuylerville': 'Schuylerville-Throgs Neck-Edgewater Park',
                  'Seagate': 'Seagate-Coney Island',
                  'Sheepshead Bay': ['Sheepshead Bay-Gerritsen Beach-Manhattan Beach','Homecrest','Madison'],
                  'Soho': 'SoHo-TriBeCa-Civic Center-Little Italy',
                  'Soundview': ['Soundview-Bruckner','West Farms-Bronx River'],
                  'South Jamaica': ['South Jamaica','Baisley Park'], 'South Ozone Park': 'South Ozone Park',
                  'South Richmond Hill': 'Richmond Hill',
                  'Springfield Gardens': ['Springfield Gardens North','Springfield Gardens South-Brookville'],
                  'St. Albans': 'St. Albans',
                  'Stuyvesant Town/PCV': 'Stuyvesant Town-Cooper Village',
                  'Sunnyside': 'Hunters Point-Sunnyside-West Maspeth',
                  'Sunset Park': 'Sunset Park East', 
                  'Throgs Neck': 'Schuylerville-Throgs Neck-Edgewater Park',
                  'Tremont': ['Fordham South','Mount Hope'],
                  'Tribeca': 'SoHo-TriBeCa-Civic Center-Little Italy',
                  'University Heights': ['University Heights-Morris Heights','Kingsbridge Heights'],
                  'Upper East Side':['Upper East Side-Carnegie Hill','Yorkville'],
                  'Upper West Side': ['Upper West Side','Lincoln Square'],
                  'Utopia': 'Fresh Meadows-Utopia',
                  'Van Nest': 'Van Nest-Morris Park-Westchester Square',
                  'Wakefield': 'Woodlawn-Wakefield',
                  'Washington Heights': ['Washington Heights North','Washington Heights South'],
                  'West Harlem': 'Manhattanville',
                  'West Village': 'West Village',
                  'Westchester Village': 'Westchester-Unionport',
                  'Whitestone': 'Whitestone',
                  'Williamsbridge': 'Williamsbridge-Olinville',
                  'Williamsburg': ['North Side-South Side','Williamsburg','East Williamsburg'],
                  'Windsor Terrace': 'Windsor Terrace',
                  'Woodhaven': 'Woodhaven',
                  'Woodlawn': 'Woodlawn-Wakefield',
                  'Woodside': 'Woodside',
                  'Woodstock': 'Melrose South-Mott Haven North'}

In [344]:
nta_neigh_dict = {}

for key, values in neigh_nta_dict.items():
    if isinstance(values, list):
        for value in values:
            if value not in nta_neigh_dict:
                nta_neigh_dict[value] = []
            nta_neigh_dict[value].append(key)
    else:
        value = values
        if value not in nta_neigh_dict:
            nta_neigh_dict[value] = []
        nta_neigh_dict[value].append(key)

print(nta_neigh_dict)

{'Astoria': ['Astoria'], 'Old Astoria': ['Astoria'], 'Steinway': ['Astoria'], 'Auburndale': ['Auburndale'], 'Bath Beach': ['Bath Beach'], 'Battery Park City-Lower Manhattan': ['Battery Park City', 'Financial District'], 'Bay Ridge': ['Bay Ridge'], 'Eastchester-Edenwald-Baychester': ['Baychester', 'Eastchester', 'Edenwald'], 'Bayside-Bayside Hills': ['Bayside'], 'Bedford Park-Fordham North': ['Bedford Park', 'Fordham'], 'Bedford': ['Bedford-Stuyvesant'], 'Stuyvesant Heights': ['Bedford-Stuyvesant'], 'Bellerose': ['Bellerose'], 'Belmont': ['Belmont'], 'Bensonhurst East': ['Bensonhurst'], 'Bensonhurst West': ['Bensonhurst'], 'Georgetown-Marine Park-Bergen Beach-Mill Basin': ['Bergen Beach', 'Marine Park', 'Mill Basin', 'Old Mill Basin'], 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill': ['Boerum Hill', 'DUMBO', 'Downtown Brooklyn'], 'Borough Park': ['Borough Park'], 'Briarwood-Jamaica Hills': ['Briarwood', 'Jamaica Hills'], 'Brighton Beach': ['Brighton Beach'], 'West Brighton': ['Bright

In [345]:
# Create a set of the filtered neighborhoods
filtered_neigh = set(neigh_salesprice.index.unique().tolist())

# Find the keys in the dictionary whose values are not in the filtered set
not_in_filtered = [key for key, value in nta_neigh_dict.items() if not set(value).issubset(filtered_neigh)]

# Print the result
print(not_in_filtered)

['Eastchester-Edenwald-Baychester', 'Bedford Park-Fordham North', 'Belmont', 'Georgetown-Marine Park-Bergen Beach-Mill Basin', 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill', 'Briarwood-Jamaica Hills', 'Bronxdale', 'Brooklyn Heights-Cobble Hill', 'Springfield Gardens South-Brookville', 'Carroll Gardens-Columbia Street-Red Hook', 'Soundview-Castle Hill-Clason Point-Harding Park', 'Midtown-Midtown South', 'Chinatown', 'Pelham Bay-Country Club-City Island', 'SoHo-TriBeCa-Civic Center-Little Italy', 'Co-op City', 'West Concourse', 'East Concourse-Concourse Village', 'Seagate-Coney Island', 'Crotona Park East', 'Flatbush', 'East Elmhurst', 'East Harlem North', 'East Harlem South', 'East Tremont', 'Glen Oaks-Floral Park-New Hyde Park', 'Fresh Meadows-Utopia', 'Sheepshead Bay-Gerritsen Beach-Manhattan Beach', 'Park Slope-Gowanus', 'Sunset Park West', 'Hamilton Heights', 'Highbridge', 'Pomonok-Flushing Heights-Hillcrest', 'Hunts Point', 'Marble Hill-Inwood', 'Van Cortlandt Village', 'Aller

In [346]:
# Create a new dictionary to store the filtered neighborhoods
new_neighborhoods_dict = {}

# Loop through each key-value pair in the original dictionary
for key, value in nta_neigh_dict.items():
    # Check if any value in the list is not in the filtered set
    filtered_values = [v for v in value if v in filtered_neigh]
    
    # Check if there are any filtered values for this key
    if filtered_values:
        # Add the key and filtered values to the new dictionary
        new_neighborhoods_dict[key] = filtered_values
        
# Update the original dictionary with the filtered dictionary
nta_neigh_dict = new_neighborhoods_dict

# Create a list of keys to delete from the dictionary
keys_to_delete = []

# Loop through each key-value pair in the updated dictionary
for key, value in nta_neigh_dict.items():
    # Check if the value list is empty
    if not value:
        # Add the key to the list of keys to delete
        keys_to_delete.append(key)

# Delete the empty keys from the dictionary
for key in keys_to_delete:
    del nta_neigh_dict[key]

In [347]:
print(nta_neigh_dict)

{'Astoria': ['Astoria'], 'Old Astoria': ['Astoria'], 'Steinway': ['Astoria'], 'Auburndale': ['Auburndale'], 'Bath Beach': ['Bath Beach'], 'Battery Park City-Lower Manhattan': ['Battery Park City', 'Financial District'], 'Bay Ridge': ['Bay Ridge'], 'Bayside-Bayside Hills': ['Bayside'], 'Bedford': ['Bedford-Stuyvesant'], 'Stuyvesant Heights': ['Bedford-Stuyvesant'], 'Bellerose': ['Bellerose'], 'Bensonhurst East': ['Bensonhurst'], 'Bensonhurst West': ['Bensonhurst'], 'Georgetown-Marine Park-Bergen Beach-Mill Basin': ['Marine Park'], 'DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill': ['Downtown Brooklyn'], 'Borough Park': ['Borough Park'], 'Briarwood-Jamaica Hills': ['Briarwood'], 'Brighton Beach': ['Brighton Beach'], 'West Brighton': ['Brighton Beach'], 'Brooklyn Heights-Cobble Hill': ['Brooklyn Heights'], 'Springfield Gardens South-Brookville': ['Springfield Gardens'], 'Brownsville': ['Brownsville'], 'Bushwick North': ['Bushwick'], 'Bushwick South': ['Bushwick'], 'Cambria Heights': ['Ca

In [351]:
neigh_nta_dict = {}

for key, values in nta_neigh_dict.items():
    if isinstance(values, list):
        for value in values:
            if value not in neigh_nta_dict:
                neigh_nta_dict[value] = []
            neigh_nta_dict[value].append(key)
    else:
        value = values
        if value not in neigh_nta_dict:
            neigh_nta_dict[value] = []
        neigh_nta_dict[value].append(key)

print(neigh_nta_dict)

{'Astoria': ['Astoria', 'Old Astoria', 'Steinway'], 'Auburndale': ['Auburndale'], 'Bath Beach': ['Bath Beach'], 'Battery Park City': ['Battery Park City-Lower Manhattan'], 'Financial District': ['Battery Park City-Lower Manhattan'], 'Bay Ridge': ['Bay Ridge'], 'Bayside': ['Bayside-Bayside Hills'], 'Bedford-Stuyvesant': ['Bedford', 'Stuyvesant Heights'], 'Bellerose': ['Bellerose'], 'Bensonhurst': ['Bensonhurst East', 'Bensonhurst West'], 'Marine Park': ['Georgetown-Marine Park-Bergen Beach-Mill Basin'], 'Downtown Brooklyn': ['DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill'], 'Borough Park': ['Borough Park'], 'Briarwood': ['Briarwood-Jamaica Hills'], 'Brighton Beach': ['Brighton Beach', 'West Brighton'], 'Brooklyn Heights': ['Brooklyn Heights-Cobble Hill'], 'Springfield Gardens': ['Springfield Gardens South-Brookville', 'Springfield Gardens North'], 'Brownsville': ['Brownsville'], 'Bushwick': ['Bushwick North', 'Bushwick South'], 'Cambria Heights': ['Cambria Heights'], 'Canarsie': ['Ca

In [304]:
# Create a new column called 'streeteasy_neighborhoods' by mapping the values in the dictionary to the 'ntaname' column
nta_nyc['streeteasy_neighborhoods'] = nta_nyc['ntaname'].map(nta_neigh_dict)
nta_nyc.head()

,boro_code,boro_name,county_fips,ntacode,ntaname,shape_leng,shape_area,geometry,streeteasy_neighborhoods
0,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ...",[St. Albans]
1,2,Bronx,005,BX28,Van Cortlandt Village,21945.719299,25666124.5948,"MULTIPOLYGON (((-73.88705 40.88435, -73.88705 ...",NaN
2,4,Queens,081,QN55,South Ozone Park,36708.1693055,82461393.7368,"MULTIPOLYGON (((-73.80577 40.68293, -73.80552 ...",[South Ozone Park]
3,3,Brooklyn,047,BK40,Windsor Terrace,19033.6720656,14041667.99,"MULTIPOLYGON (((-73.98017 40.66115, -73.98021 ...",[Windsor Terrace]
4,3,Brooklyn,047,BK50,Canarsie,43703.6096661,82089678.6389,"MULTIPOLYGON (((-73.88834 40.64671, -73.88835 ...",[Canarsie]


In [306]:
nta_filtered = nta_nyc[~nta_nyc['streeteasy_neighborhoods'].isna()]
print(len(nta_filtered))

127


In [307]:
def clean_neighborhoods(neigh_list):
    if len(neigh_list) == 1:
        return neigh_list[0]
    else:
        return neigh_list

nta_filtered['streeteasy_neighborhoods'] = nta_filtered['streeteasy_neighborhoods'].apply(clean_neighborhoods)
nta_filtered.head(15)

,boro_code,boro_name,county_fips,ntacode,ntaname,shape_leng,shape_area,geometry,streeteasy_neighborhoods
0,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ...",St. Albans
2,4,Queens,081,QN55,South Ozone Park,36708.1693055,82461393.7368,"MULTIPOLYGON (((-73.80577 40.68293, -73.80552 ...",South Ozone Park
3,3,Brooklyn,047,BK40,Windsor Terrace,19033.6720656,14041667.99,"MULTIPOLYGON (((-73.98017 40.66115, -73.98021 ...",Windsor Terrace
4,3,Brooklyn,047,BK50,Canarsie,43703.6096661,82089678.6389,"MULTIPOLYGON (((-73.88834 40.64671, -73.88835 ...",Canarsie
6,4,Queens,081,QN26,North Corona,18429.5400774,18003888.327,"MULTIPOLYGON (((-73.85450 40.75828, -73.85421 ...",North Corona
7,3,Brooklyn,047,BK68,Fort Greene,19825.5208594,16482821.6487,"MULTIPOLYGON (((-73.96939 40.69608, -73.96938 ...",Fort Greene
8,3,Brooklyn,047,BK79,Ocean Hill,22494.5812446,20089910.6584,"MULTIPOLYGON (((-73.90321 40.67874, -73.90256 ...",Crown Heights
10,4,Queens,081,QN52,East Flushing,25848.5454461,29453683.7952,"MULTIPOLYGON (((-73.79493 40.75780, -73.79475 ...",Flushing
11,3,Brooklyn,047,BK44,Madison,26237.2576492,27379162.0595,"MULTIPOLYGON (((-73.93754 40.60856, -73.93830 ...",Sheepshead Bay
12,4,Queens,081,QN01,South Jamaica,32122.552258,39971928.7079,"MULTIPOLYGON (((-73.77396 40.69981, -73.77446 ...",South Jamaica


In [232]:
nta_filtered.dtypes

boro_code                     object
boro_name                     object
county_fips                   object
ntacode                       object
ntaname                       object
shape_leng                    object
shape_area                    object
geometry                    geometry
streeteasy_neighborhoods      object
dtype: object

In [399]:
#import ts data
timeseries_311 = pd.read_csv('../Data/311_monthlycomplaints_nta_agency.csv')
timeseries_311.head(10)

,ntacode,date,3-1-1 Call Center,Correspondence Unit,Department for the Aging,Department of Buildings,Department of Consumer Affairs,Department of Education,Department of Environmental Protection,Department of Health and Mental Hygiene,...,Disability Rent Increase Exemption Unit,Commercial Exemption Unit,DRIE Unit,Traffic Management Center,Department of Homeless Services,Discrepancy and Billing,External Affairs Unit,Payment Operations,Mayorâs Office of Special Enforcement,Office of Technology and Innovation
0,BK09,2010-01-01,1.0,0.0,0.0,23.0,2.0,0.0,34.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BK17,2010-01-01,0.0,0.0,0.0,78.0,10.0,2.0,127.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BK19,2010-01-01,0.0,0.0,0.0,45.0,8.0,2.0,72.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BK21,2010-01-01,0.0,0.0,1.0,31.0,1.0,0.0,64.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BK23,2010-01-01,0.0,0.0,0.0,6.0,5.0,1.0,6.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,BK25,2010-01-01,0.0,0.0,0.0,31.0,6.0,0.0,95.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,BK26,2010-01-01,0.0,0.0,0.0,30.0,2.0,0.0,64.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,BK27,2010-01-01,0.0,0.0,0.0,30.0,2.0,0.0,26.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,BK28,2010-01-01,0.0,0.0,0.0,91.0,14.0,2.0,61.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,BK29,2010-01-01,0.0,0.0,0.0,69.0,4.0,1.0,44.0,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [400]:
timeseries_311.columns

Index(['ntacode', 'date', '3-1-1 Call Center', 'Correspondence Unit',
       'Department for the Aging', 'Department of Buildings',
       'Department of Consumer Affairs', 'Department of Education',
       'Department of Environmental Protection',
       'Department of Health and Mental Hygiene',
       'Department of Housing Preservation and Development',
       'Department of Information Technology and Telecommunications',
       'Department of Parks and Recreation', 'Department of Sanitation',
       'Department of Transportation', 'NYC Public Schools',
       'New York City Police Department', 'Refunds and Adjustments',
       'Taxi and Limousine Commission', 'Economic Development Corporation',
       'Division of Alternative Management', 'HRA Benefit Card Replacement',
       'Child Care and Camps Complaint Unit', 'NYC Emergency Management',
       'Mosquito Control', 'Office of the Taxpayer Advocate',
       'Disability Rent Increase Exemption Unit', 'Commercial Exemption Unit',

In [401]:
# select the rows in nta_filtered where 'streeteasy_neighborhoods' only has 1 unique item
nta_filtered_unique = nta_filtered[nta_filtered['streeteasy_neighborhoods'].apply(lambda x: isinstance(x, str))]
nta_filtered_unique['ntacode'] = nta_filtered_unique['ntacode'].astype(str)
timeseries_311['ntacode'] = timeseries_311['ntacode'].astype(str)
# join the two dataframes on the 'ntacode' index
joined_df = pd.merge(nta_filtered_unique,timeseries_311,on = 'ntacode', how = 'inner')
joined_df.head()

,boro_code,boro_name,county_fips,ntacode,ntaname,shape_leng,shape_area,geometry,streeteasy_neighborhoods,date,...,Disability Rent Increase Exemption Unit,Commercial Exemption Unit,DRIE Unit,Traffic Management Center,Department of Homeless Services,Discrepancy and Billing,External Affairs Unit,Payment Operations,Mayorâs Office of Special Enforcement,Office of Technology and Innovation
0,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ...",St. Albans,2010-01-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ...",St. Albans,2010-02-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ...",St. Albans,2010-03-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ...",St. Albans,2010-04-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ...",St. Albans,2010-05-01,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [402]:
nta_filtered_unique.sort_values(by = 'streeteasy_neighborhoods').head(10)

,boro_code,boro_name,county_fips,ntacode,ntaname,shape_leng,shape_area,geometry,streeteasy_neighborhoods
120,4,Queens,081,QN71,Old Astoria,19367.6561995,15620456.0782,"MULTIPOLYGON (((-73.92828 40.77690, -73.92785 ...",Astoria
136,4,Queens,081,QN70,Astoria,32536.0164884,39335039.1473,"MULTIPOLYGON (((-73.90203 40.76742, -73.90218 ...",Astoria
137,4,Queens,081,QN72,Steinway,43564.4355845,57725501.9839,"MULTIPOLYGON (((-73.89183 40.77488, -73.89034 ...",Astoria
15,4,Queens,081,QN48,Auburndale,32451.7673262,34167059.6366,"MULTIPOLYGON (((-73.77574 40.74333, -73.77580 ...",Auburndale
66,3,Brooklyn,047,BK27,Bath Beach,22422.3702463,20556460.0272,"MULTIPOLYGON (((-74.00204 40.59406, -74.00228 ...",Bath Beach
107,3,Brooklyn,047,BK31,Bay Ridge,44542.2764965,67226641.484,"MULTIPOLYGON (((-74.03230 40.64358, -74.02983 ...",Bay Ridge
37,4,Queens,081,QN46,Bayside-Bayside Hills,42574.9103944,80744707.8409,"MULTIPOLYGON (((-73.76760 40.77830, -73.76762 ...",Bayside
183,3,Brooklyn,047,BK35,Stuyvesant Heights,28239.0853295,31407114.0944,"MULTIPOLYGON (((-73.91741 40.68403, -73.91727 ...",Bedford-Stuyvesant
18,3,Brooklyn,047,BK75,Bedford,29992.9240956,32629788.7441,"MULTIPOLYGON (((-73.94193 40.70073, -73.94372 ...",Bedford-Stuyvesant
78,4,Queens,081,QN43,Bellerose,32086.8007951,55193394.6222,"MULTIPOLYGON (((-73.71050 40.72723, -73.71056 ...",Bellerose


In [403]:
# keep only the 'streeteasy_neighborhoods' column from the nta_filtered dataframe
final_df_311 = joined_df[timeseries_311.columns.tolist()].dropna()
final_df_311['date'] = pd.to_datetime(final_df_311['date'], format='%Y-%m-%d')

# Format the date column as 'yyyy-mm'
final_df_311['date'] = final_df_311['date'].dt.strftime('%Y-%m')

# Convert the date column back to an object datatype
final_df_311['date'] = final_df_311['date'].astype('object')
final_df_311.head()

,ntacode,date,3-1-1 Call Center,Correspondence Unit,Department for the Aging,Department of Buildings,Department of Consumer Affairs,Department of Education,Department of Environmental Protection,Department of Health and Mental Hygiene,...,Disability Rent Increase Exemption Unit,Commercial Exemption Unit,DRIE Unit,Traffic Management Center,Department of Homeless Services,Discrepancy and Billing,External Affairs Unit,Payment Operations,Mayorâs Office of Special Enforcement,Office of Technology and Innovation
0,QN08,2010-01,0.0,0.0,2.0,73.0,17.0,2.0,146.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,QN08,2010-02,0.0,0.0,0.0,54.0,4.0,0.0,93.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,QN08,2010-03,0.0,0.0,0.0,65.0,2.0,2.0,164.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,QN08,2010-04,0.0,0.0,0.0,58.0,9.0,1.0,154.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,QN08,2010-05,0.0,0.0,0.0,60.0,7.0,0.0,131.0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [363]:
# join the two dataframes on the 'ntacode' index
joined_df = pd.merge(nta_filtered_unique,neigh_salesprice,left_on = 'streeteasy_neighborhoods',right_on = 'areaName', how = 'inner')
joined_df.head()

,boro_code,boro_name,county_fips,ntacode,ntaname,shape_leng,shape_area,geometry,streeteasy_neighborhoods,areaName,...,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
0,4,Queens,081,QN08,St. Albans,45401.316803,77412747.847,"MULTIPOLYGON (((-73.75205 40.70523, -73.75174 ...",St. Albans,St. Albans,...,655000.0,680600.0,655000.0,622500.0,610000.0,589999.0,605222.0,570000.0,583871.0,580000.0
1,4,Queens,081,QN55,South Ozone Park,36708.1693055,82461393.7368,"MULTIPOLYGON (((-73.80577 40.68293, -73.80552 ...",South Ozone Park,South Ozone Park,...,635000.0,672500.0,675000.0,742500.0,765000.0,700000.0,666250.0,715000.0,0.0,0.0
2,3,Brooklyn,047,BK40,Windsor Terrace,19033.6720656,14041667.99,"MULTIPOLYGON (((-73.98017 40.66115, -73.98021 ...",Windsor Terrace,Windsor Terrace,...,1150500.0,1389400.0,1025000.0,0.0,895000.0,0.0,0.0,840000.0,0.0,0.0
3,3,Brooklyn,047,BK50,Canarsie,43703.6096661,82089678.6389,"MULTIPOLYGON (((-73.88834 40.64671, -73.88835 ...",Canarsie,Canarsie,...,670000.0,657500.0,725000.0,670000.0,750000.0,690000.0,631000.0,754898.5,689000.0,797500.0
4,4,Queens,081,QN26,North Corona,18429.5400774,18003888.327,"MULTIPOLYGON (((-73.85450 40.75828, -73.85421 ...",North Corona,North Corona,...,1045000.0,650566.0,795000.0,0.0,0.0,0.0,0.0,0.0,800000.0,0.0


In [365]:
joined_df.sort_values(by = 'areaName').head(10)

,boro_code,boro_name,county_fips,ntacode,ntaname,shape_leng,shape_area,geometry,streeteasy_neighborhoods,areaName,...,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03
88,4,Queens,081,QN71,Old Astoria,19367.6561995,15620456.0782,"MULTIPOLYGON (((-73.92828 40.77690, -73.92785 ...",Astoria,Astoria,...,936776.0,883500.0,620000.0,810000.0,1049000.0,999000.0,980000.0,950000.0,1150000.0,927500.0
89,4,Queens,081,QN70,Astoria,32536.0164884,39335039.1473,"MULTIPOLYGON (((-73.90203 40.76742, -73.90218 ...",Astoria,Astoria,...,936776.0,883500.0,620000.0,810000.0,1049000.0,999000.0,980000.0,950000.0,1150000.0,927500.0
90,4,Queens,081,QN72,Steinway,43564.4355845,57725501.9839,"MULTIPOLYGON (((-73.89183 40.77488, -73.89034 ...",Astoria,Astoria,...,936776.0,883500.0,620000.0,810000.0,1049000.0,999000.0,980000.0,950000.0,1150000.0,927500.0
19,4,Queens,081,QN48,Auburndale,32451.7673262,34167059.6366,"MULTIPOLYGON (((-73.77574 40.74333, -73.77580 ...",Auburndale,Auburndale,...,912500.0,940000.0,1053000.0,990000.0,925000.0,982500.0,910000.0,915000.0,0.0,0.0
58,3,Brooklyn,047,BK27,Bath Beach,22422.3702463,20556460.0272,"MULTIPOLYGON (((-74.00204 40.59406, -74.00228 ...",Bath Beach,Bath Beach,...,852500.0,1052500.0,1112500.0,744500.0,1050000.0,0.0,0.0,1085000.0,0.0,0.0
82,3,Brooklyn,047,BK31,Bay Ridge,44542.2764965,67226641.484,"MULTIPOLYGON (((-74.03230 40.64358, -74.02983 ...",Bay Ridge,Bay Ridge,...,765000.0,999000.0,599500.0,650000.0,625000.0,667000.0,833946.0,996866.0,435000.0,900000.0
38,4,Queens,081,QN46,Bayside-Bayside Hills,42574.9103944,80744707.8409,"MULTIPOLYGON (((-73.76760 40.77830, -73.76762 ...",Bayside,Bayside,...,780500.0,769988.0,830000.0,957500.0,737000.0,818000.0,770000.0,555000.0,705000.0,500000.0
24,3,Brooklyn,047,BK35,Stuyvesant Heights,28239.0853295,31407114.0944,"MULTIPOLYGON (((-73.91741 40.68403, -73.91727 ...",Bedford-Stuyvesant,Bedford-Stuyvesant,...,1065000.0,1126250.0,1173450.0,999999.0,1195000.0,1255250.0,900000.0,994950.0,1115000.0,1033896.5
23,3,Brooklyn,047,BK75,Bedford,29992.9240956,32629788.7441,"MULTIPOLYGON (((-73.94193 40.70073, -73.94372 ...",Bedford-Stuyvesant,Bedford-Stuyvesant,...,1065000.0,1126250.0,1173450.0,999999.0,1195000.0,1255250.0,900000.0,994950.0,1115000.0,1033896.5
68,4,Queens,081,QN43,Bellerose,32086.8007951,55193394.6222,"MULTIPOLYGON (((-73.71050 40.72723, -73.71056 ...",Bellerose,Bellerose,...,700000.0,757000.0,630000.0,670000.0,0.0,690000.0,0.0,760000.0,640000.0,0.0


In [367]:
# keep only the 'streeteasy_neighborhoods' column from the nta_filtered dataframe
final_df_sales = joined_df[['ntacode']+neigh_salesprice.columns.tolist()[1:-3]].dropna()
final_df_sales.head()

,ntacode,2010-01,2010-02,2010-03,2010-04,2010-05,2010-06,2010-07,2010-08,2010-09,...,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
0,QN08,285000.0,250000.0,312750.0,340000.0,349800.0,324300.0,310358.0,296960.0,300000.0,...,690000.0,655000.0,620000.0,655000.0,680600.0,655000.0,622500.0,610000.0,589999.0,605222.0
1,QN55,340000.0,375000.0,320750.0,320000.0,353386.0,328512.5,314000.0,334750.0,289200.0,...,624000.0,712500.0,686000.0,635000.0,672500.0,675000.0,742500.0,765000.0,700000.0,666250.0
2,BK40,308500.0,590000.0,0.0,0.0,0.0,400000.0,715000.0,575000.0,455750.0,...,900000.0,710000.0,1050000.0,1150500.0,1389400.0,1025000.0,0.0,895000.0,0.0,0.0
3,BK50,444575.0,385000.0,410000.0,415000.0,415000.0,436000.0,329000.0,425000.0,421500.0,...,781000.0,744500.0,730000.0,670000.0,657500.0,725000.0,670000.0,750000.0,690000.0,631000.0
4,QN26,582250.0,363000.0,415000.0,420000.0,475000.0,607500.0,522500.0,380000.0,505500.0,...,1012500.0,0.0,830000.0,1045000.0,650566.0,795000.0,0.0,0.0,0.0,0.0


In [392]:
# Use the pandas melt function to reshape the dataframe
df = pd.melt(final_df_sales, id_vars=['ntacode'], var_name='date', value_name='median_saleprice')

# Optional: Convert the date column from string to datetime format
# df['date'] = pd.to_datetime(df['date'], format='%Y-%m')

# Optional: Sort the dataframe by ntacode and date
df = df.sort_values(['ntacode', 'date'])

# Print the resulting dataframe
df.head(20)

,ntacode,date,median_saleprice
44,BK09,2010-01,795000.0
164,BK09,2010-02,722500.0
284,BK09,2010-03,585000.0
404,BK09,2010-04,562473.0
524,BK09,2010-05,660000.0
644,BK09,2010-06,666250.0
764,BK09,2010-07,865000.0
884,BK09,2010-08,770000.0
1004,BK09,2010-09,605858.0
1124,BK09,2010-10,557000.0


In [393]:
df[df['ntacode'] == 'QN08']

,ntacode,date,median_saleprice
0,QN08,2010-01,285000.0
120,QN08,2010-02,250000.0
240,QN08,2010-03,312750.0
360,QN08,2010-04,340000.0
480,QN08,2010-05,349800.0
...,...,...,...
18120,QN08,2022-08,655000.0
18240,QN08,2022-09,622500.0
18360,QN08,2022-10,610000.0
18480,QN08,2022-11,589999.0


In [394]:
df.dtypes

ntacode              object
date                 object
median_saleprice    float64
dtype: object

In [404]:
final_df_311.dtypes

ntacode                                                         object
date                                                            object
3-1-1 Call Center                                              float64
Correspondence Unit                                            float64
Department for the Aging                                       float64
Department of Buildings                                        float64
Department of Consumer Affairs                                 float64
Department of Education                                        float64
Department of Environmental Protection                         float64
Department of Health and Mental Hygiene                        float64
Department of Housing Preservation and Development             float64
Department of Information Technology and Telecommunications    float64
Department of Parks and Recreation                             float64
Department of Sanitation                                       float64
Depart

In [405]:
final_outputdata = pd.merge(final_df_311,df,how = 'left',on = ['ntacode','date'])
final_outputdata.tail(25)

,ntacode,date,3-1-1 Call Center,Correspondence Unit,Department for the Aging,Department of Buildings,Department of Consumer Affairs,Department of Education,Department of Environmental Protection,Department of Health and Mental Hygiene,...,Commercial Exemption Unit,DRIE Unit,Traffic Management Center,Department of Homeless Services,Discrepancy and Billing,External Affairs Unit,Payment Operations,Mayorâs Office of Special Enforcement,Office of Technology and Innovation,median_saleprice
18695,QN53,2020-12,0.0,0.0,0.0,56.0,4.0,0.0,59.0,9.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,16.0,0.0,665000.0
18696,QN53,2021-01,0.0,0.0,0.0,51.0,6.0,0.0,67.0,12.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,20.0,0.0,666000.0
18697,QN53,2021-02,0.0,0.0,0.0,46.0,5.0,0.0,26.0,12.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,6.0,0.0,810500.0
18698,QN53,2021-03,0.0,0.0,0.0,52.0,4.0,0.0,62.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,638000.0
18699,QN53,2021-04,0.0,0.0,0.0,60.0,13.0,0.0,82.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,607500.0
18700,QN53,2021-05,0.0,0.0,0.0,64.0,9.0,0.0,99.0,17.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,699000.0
18701,QN53,2021-06,0.0,0.0,0.0,69.0,11.0,1.0,113.0,29.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,675000.0
18702,QN53,2021-07,0.0,0.0,0.0,63.0,7.0,0.0,117.0,17.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,660000.0
18703,QN53,2021-08,0.0,0.0,0.0,65.0,11.0,0.0,76.0,24.0,...,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,702500.0
18704,QN53,2021-09,0.0,0.0,0.0,77.0,8.0,1.0,154.0,24.0,...,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,690000.0


In [406]:
final_outputdata.to_csv('../Data/final_311_comps_saleprice_data.csv')

In [3]:
final_outputdata = pd.read_csv('../Data/final_311_comps_saleprice_data.csv',index_col = 0)
final_outputdata.head()

,ntacode,date,3-1-1 Call Center,Correspondence Unit,Department for the Aging,Department of Buildings,Department of Consumer Affairs,Department of Education,Department of Environmental Protection,Department of Health and Mental Hygiene,...,Commercial Exemption Unit,DRIE Unit,Traffic Management Center,Department of Homeless Services,Discrepancy and Billing,External Affairs Unit,Payment Operations,Mayorâs Office of Special Enforcement,Office of Technology and Innovation,median_saleprice
0,QN08,2010-01,0.0,0.0,2.0,73.0,17.0,2.0,146.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,285000.0
1,QN08,2010-02,0.0,0.0,0.0,54.0,4.0,0.0,93.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250000.0
2,QN08,2010-03,0.0,0.0,0.0,65.0,2.0,2.0,164.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,312750.0
3,QN08,2010-04,0.0,0.0,0.0,58.0,9.0,1.0,154.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,340000.0
4,QN08,2010-05,0.0,0.0,0.0,60.0,7.0,0.0,131.0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,349800.0


In [4]:
len(final_outputdata.ntacode.unique())

120

In [5]:
len(final_outputdata.columns.tolist()[2:])

35